<h1><center>Полиномиальная регрессия</center></h1>

<h4>About this Notebook</h4>
В этом блокноте мы узнаем, как реализовать полиномиальную регрессию. Мы загрузим набор данных, который связан с расходом топлива и выбросами углекислого газа автомобилей. Затем мы разделим наши данные на обучающие и тестовые наборы, создадим модель с использованием обучающего набора, оценим нашу модель с использованием тестового набора и, наконец, используем модель для прогнозирования неизвестного значения.

### Importing Needed packages

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline

## Reading the data in

In [ ]:
df = pd.read_csv("FuelConsumption.csv")

# take a look at the dataset
df.head()

Выберем некоторые показатели, которые мы хотим использовать для регрессии.

In [ ]:
cdf = df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB','CO2EMISSIONS']]
cdf.head(9)

Построим значения выбросов в зависимости от объема двигателя:

In [ ]:
plt.scatter(cdf.ENGINESIZE, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

#### Создание обучающего и тестового набора данных
Разделение обучения и теста включает в себя разделение набора данных на наборы обучения и тестирования, которые являются взаимоисключающими. После этого вы обучаете с тренировочным набором и тестируете с тестовым набором.

In [ ]:
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]

<h2 id="polynomial_regression">Полиномиальная регрессия</h2>

Иногда тенденция данных не является действительно линейной и выглядит нелинейной. В этом случае мы можем использовать методы полиномиальной регрессии. На самом деле существует множество различных регрессий, которые можно использовать для подбора любого набора данных, такие как квадратичная, кубическая и т. д.

По сути, мы можем назвать все это полиномиальной регрессией, где связь между независимой переменной x и зависимой переменной y моделируется как полином n-й степени по x. Допустим, вы хотите иметь полиномиальную регрессию (давайте сделаем полином 2 степени):


$y = b + \theta_1  x + \theta_2 x^2$

Теперь возникает вопрос: как мы можем подогнать наши данные к этому уравнению, в то время как у нас есть только значения x, такие как __Engine Size__?
Мы можем создать несколько дополнительных переменных: 1, $x$, and $x^2$.


Функция __PloynomialFeatures()__ в библиотеке sklearn запускает новые наборы показателей из исходного набора показателей. То есть будет сгенерирована матрица, состоящая из всех полиномиальных комбинаций признаков со степенью, меньшей или равной указанной степени. Например, предположим, что исходный набор показателей имеет только один показатель, _ENGINESIZE_. Теперь, если мы выберем степень многочлена равной 2, то он генерирует 3 элемента: степень = 0, степень = 1 и степень = 2:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
train_x = np.asanyarray(train[['ENGINESIZE']])
train_y = np.asanyarray(train[['CO2EMISSIONS']])

test_x = np.asanyarray(test[['ENGINESIZE']])
test_y = np.asanyarray(test[['CO2EMISSIONS']])


poly = PolynomialFeatures(degree=2)
train_x_poly = poly.fit_transform(train_x)
train_x_poly

**fit_transform** принимает наши значения x и выводит список наших данных, возведенных в степень от 0 до степени 2 (поскольку мы установили степень нашего полинома как 2).

$
\begin{bmatrix}
    v_1\\
    v_2\\
    \vdots\\
    v_n
\end{bmatrix}
$
$\longrightarrow$
$
\begin{bmatrix}
    [ 1 & v_1 & v_1^2]\\
    [ 1 & v_2 & v_2^2]\\
    \vdots & \vdots & \vdots\\
    [ 1 & v_n & v_n^2]
\end{bmatrix}
$

in our example

$
\begin{bmatrix}
    2.\\
    2.4\\
    1.5\\
    \vdots
\end{bmatrix}
$
$\longrightarrow$
$
\begin{bmatrix}
    [ 1 & 2. & 4.]\\
    [ 1 & 2.4 & 5.76]\\
    [ 1 & 1.5 & 2.25]\\
    \vdots & \vdots & \vdots\\
\end{bmatrix}
$

Полиномиальная регрессия - это особый случай линейной регрессии, основная идея которой состоит в том, как выбрать показатели. Просто подумайте о замене $ x $ на $ x_1 $, $ x_1 ^ 2 $ на $ x_2 $ и так далее. Тогда уравнение степени 2 будет преобразовано в:

$y = b + \theta_1  x_1 + \theta_2 x_2$

Теперь мы можем рассматривать ее как задачу «линейной регрессии». Следовательно, эта полиномиальная регрессия считается частным случаем традиционной множественной линейной регрессии. Таким образом, вы можете использовать тот же механизм, что и для линейной регрессии.


так что мы можем использовать функцию __LinearRegression()__ для решения:

In [ ]:
clf = linear_model.LinearRegression()
train_y_ = clf.fit(train_x_poly, train_y)
# The coefficients
print ('Coefficients: ', clf.coef_)
print ('Intercept: ',clf.intercept_)

Как уже упоминалось ранее, __Coefficient__ и __Intercept__, это параметры кривой соответствия.
Учитывая, что это типичная множественная линейная регрессия, с 3 параметрами, и зная, что параметры являются пересечением и коэффициентами гиперплоскости, sklearn оценивает их по нашему новому набору показателей.

In [ ]:
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue')
XX = np.arange(0.0, 10.0, 0.1)
yy = clf.intercept_[0]+ clf.coef_[0][1]*XX+ clf.coef_[0][2]*np.power(XX, 2)
plt.plot(XX, yy, '-r' )
plt.xlabel("Engine size")
plt.ylabel("Emission")

<h2 id="evaluation">Оценка</h2>

In [ ]:
from sklearn.metrics import r2_score

test_x_poly = poly.fit_transform(test_x)
test_y_ = clf.predict(test_x_poly)

print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y_ , test_y) )

<h2 id="practice">Практика</h2>
Попробуйте использовать полиномиальную регрессию с набором данных, но на этот раз со степенью три (куб.). Это приводит к лучшей точности?

In [ ]:
# write your code here


Дважды щелкните __здесь__ чтобы посмотреть решение.

<!-- Your answer is below:

poly3 = PolynomialFeatures(degree=3)
train_x_poly3 = poly3.fit_transform(train_x)
clf3 = linear_model.LinearRegression()
train_y3_ = clf3.fit(train_x_poly3, train_y)
# The coefficients
print ('Coefficients: ', clf3.coef_)
print ('Intercept: ',clf3.intercept_)
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue')
XX = np.arange(0.0, 10.0, 0.1)
yy = clf3.intercept_[0]+ clf3.coef_[0][1]*XX + clf3.coef_[0][2]*np.power(XX, 2) + clf3.coef_[0][3]*np.power(XX, 3)
plt.plot(XX, yy, '-r' )
plt.xlabel("Engine size")
plt.ylabel("Emission")
test_x_poly3 = poly3.fit_transform(test_x)
test_y3_ = clf3.predict(test_x_poly3)
print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y3_ - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y3_ - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y3_ , test_y) )


-->